In [1]:
import snap
import numpy

TPATH = "/courses/TSKS33/ht2023/data/"
def genmod10star():   
    #G1 = snap.GenPrefAttach(1000, 10)
    #G1 = snap.GenRndPowerLaw(100000,2.2)
    G = snap.GenStar(snap.PUNGraph,10,False)
    G.AddEdge(4,5)
    #G = snap.GetMxScc(G1)
    N = G.GetNodes()
    
    # node numbering in the mat file is sequential n=1,2,... following the node iterator
    # gio.h line 369
#    snap.SaveMatlabSparseMtx(G, "mod10star.mat")

    # assign the degree as attribute
    x = snap.TIntFltH(N)
#    file = open("mod10star_attr.mat",'w') 
    for NI in G.Nodes():
        n=NI.GetId()
        x[n]=NI.GetDeg()
#        file.write(str(n+1)+"\t"+str(x[n])+"\n")
#        file.write(str(x[n])+"\n")
#    file.close()
    
    return G, x

In [2]:
G, x = genmod10star()

sum = 0
for n in G.Nodes():
    sum+=n.GetDeg()
exact = sum/G.GetNodes()
print(exact)

2.0


In [5]:
S = 10**4
S

10000

In [14]:
import random
num_runs = 5
sums = []
for i in range(num_runs):
    random.seed(i)
    sampled_nodes = random.choices(list(G.Nodes()), k=S)
    tmp_sum = 0
    for node in sampled_nodes:
        n = G.GetNodes()
        print(node.GetDeg())
        tmp_sum += node.GetDeg()
    sums.append(tmp_sum/S)
print(sums)




TypeError: object of type 'generator' has no len()